In [44]:
from __future__ import print_function
import pandas as pd
import numpy as np

df = pd.read_excel("CHOPER_IBAQ_sep.xlsx",header=0,index_col=0)
df.loc["Day",:] = df.loc["Day",:].apply(int)
# Replace 0 with NaN



In [49]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

days = []
for name,values in df.iterrows():
    if name == "Day":
        days = values.apply(int)
        continue
    analys_df = pd.DataFrame({"Day": days.values, "Expression" : values.values},index=df.columns)
    analys_df.drop(analys_df.loc[(analys_df.loc[:,"Expression"]==0.0)].index,inplace=True,axis=0)
    model = ols("Expression ~ C(Day)",analys_df).fit()
    report = anova_lm(model,type=2)
    print(model.params) 
    

Intercept    266360.0
dtype: float64
Intercept    153490.0
dtype: float64
Intercept       2.830533e+09
C(Day)[T.5]    -3.543333e+07
C(Day)[T.10]   -2.173203e+09
C(Day)[T.12]   -2.011477e+09
C(Day)[T.14]   -2.046640e+09
C(Day)[T.18]   -1.959287e+09
C(Day)[T.22]    7.042333e+08
C(Day)[T.23]    1.416067e+09
C(Day)[T.26]    4.052333e+08
C(Day)[T.27]   -2.066333e+08
C(Day)[T.28]   -1.209910e+09
C(Day)[T.30]   -1.594050e+09
C(Day)[T.38]    4.941000e+08
C(Day)[T.39]   -9.246333e+08
C(Day)[T.40]   -7.551000e+08
C(Day)[T.41]   -5.384667e+08
C(Day)[T.45]   -8.067333e+08
dtype: float64
Intercept       4.635367e+06
C(Day)[T.5]     1.685300e+06
C(Day)[T.10]   -3.170743e+06
C(Day)[T.22]   -3.949790e+06
C(Day)[T.27]   -4.283437e+06
C(Day)[T.28]   -4.313430e+06
C(Day)[T.30]    7.301000e+05
C(Day)[T.41]   -4.246192e+06
dtype: float64
Intercept       7.462833e+06
C(Day)[T.5]     7.096167e+06
C(Day)[T.10]   -4.211200e+06
C(Day)[T.14]   -7.333063e+06
C(Day)[T.22]   -5.751733e+06
C(Day)[T.23]   -7.169113e+

Intercept       3.498233e+06
C(Day)[T.5]     9.490667e+05
C(Day)[T.22]   -1.979933e+06
C(Day)[T.23]   -2.658917e+06
C(Day)[T.26]   -2.243500e+06
C(Day)[T.27]   -2.360900e+06
C(Day)[T.28]   -2.734548e+06
C(Day)[T.30]   -2.927003e+06
C(Day)[T.38]   -2.516580e+06
C(Day)[T.39]   -2.637358e+06
C(Day)[T.40]   -2.832923e+06
C(Day)[T.45]   -2.755907e+06
dtype: float64
Intercept       2.732933e+05
C(Day)[T.5]     2.633333e+02
C(Day)[T.10]    3.762667e+05
C(Day)[T.12]    1.211067e+06
C(Day)[T.14]    5.747767e+05
C(Day)[T.18]    1.700140e+06
C(Day)[T.22]    1.943240e+06
C(Day)[T.23]    1.822263e+06
C(Day)[T.26]    1.187030e+06
C(Day)[T.27]    1.799713e+06
C(Day)[T.28]    8.735933e+05
C(Day)[T.30]    2.462707e+06
C(Day)[T.38]    1.581537e+06
C(Day)[T.39]    2.146073e+06
C(Day)[T.40]    1.654973e+06
C(Day)[T.41]    1.792007e+06
C(Day)[T.45]    1.594820e+06
dtype: float64
Intercept      62481.666667
C(Day)[T.5]    43336.333333
dtype: float64
Intercept      142220.000000
C(Day)[T.5]     -4582.333333


ValueError: zero-size array to reduction operation maximum which has no identity

In [46]:
dir(statsmodels.tools.tools)

['Bunch',
 'L',
 '__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '_ensure_2d',
 '_is_recarray',
 '_is_using_pandas',
 '_make_dictnames',
 'add_constant',
 'asstr2',
 'categorical',
 'chain_dot',
 'clean0',
 'drop_missing',
 'fullrank',
 'isestimable',
 'lmap',
 'long',
 'lzip',
 'maybe_unwrap_results',
 'nan_dot',
 'np',
 'np_matrix_rank',
 'nprf',
 'pd',
 'pinv_extended',
 'range',
 'rank',
 'recipr',
 'recipr0',
 'reduce',
 'svdvals',
 'unsqueeze',
 'webuse']